This file was executed on Colab due to large model sizes

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/sentence-t5-xxl').to("cuda")


In [ ]:
from logging import exception
import numpy as np
import os
import pandas as pd
import time
import re

#Create text dataset

os.makedirs("text-embeddings-short-t5-xxl", exist_ok=True)

for folder in os.listdir("/content/drive/MyDrive/new"):
    if folder.startswith("."):
      continue
    person = folder.strip("P")

    for file in os.listdir(f"/content/drive/MyDrive/new/{folder}"):
      if not file.endswith(".xlsx"):
        continue

      X = []
      y = []
      turn = []
      x_contents = []
      y_contents = []
      print(f"Processing file: {file} for person {person}")
      file_path = os.path.join(f"/content/drive/MyDrive/new/{folder}", file)
      df = pd.read_excel(file_path)
      df = df.dropna()
      for i, row in df.iterrows():
          content=row["Exchange text"]
          user_utterances = re.findall(r"User:\s*(.*?)(?=\s*(Robot:|User:)|$)", content)
          cleaned = ("").join([text.strip() for text, _ in user_utterances])
          if cleaned == "":
              continue
          x_contents.append(cleaned)
          y_contents.append(int(row["Rating"]))
          turn.append(int(row["Turn ID"]))
      # Generate embeddings for the text data
      print(x_contents)
      while True:
        try:
          results = model.encode(x_contents)
          break
        except Exception as e:
          print(e)
          time.sleep(1)

      # Convert to numpy arrays
      X = [result for result in results]
      X = np.array(X)
      y = np.array(y_contents)
      turn = np.array(turn)
      # Save the dataset to a CSV file with pandas
      dataset = pd.DataFrame()
      dataset["Turn ID"] = turn
      dataset = pd.concat([dataset, pd.DataFrame(X)], axis=1)
      # print(dataset)
      # dataset["Rating"] = y
      dataset.to_csv(f"text-embeddings-short-t5-xxl/text-dataset-P{person}.csv", index=False)
      print(f"Processed {file} with {len(df)} entries.")